# Accessing generated particles to calculate production fractions

In this example, we access the container of generated particles (`genParticles`) and count the number of different $B$-hadron species produced in the $Z^0 \to b\bar{b}$ hadronisation.

In [1]:
import uproot4 as uproot
import awkward1 as ak
import sys,os
import json
import numpy as np
from fcc_python_tools.locations import loc
from particle import literals as lp

Let's load the `ROOT` file with `uproot` and access the `events` tree:

In [2]:
#file = uproot.open("/eos/experiment/fcc/ee/tmp/fcc_python_tools/FCCDelphesOutput_10k.root")
file = uproot.open("/eos/experiment/fcc/ee/tmp/FCCDelphesOutput_Bu2D0Pi_qq.root")
tree = file['events']

Next we load the `genParticles` container into an `awkward array`, using the wildcard `*` to load all branches.

In [3]:
gen_container = "genParticles.core"
gen = tree.arrays(filter_name=f"{gen_container}*",how="zip")

There is a useful package called [particle](https://github.com/scikit-hep/particle) which contains information on PDG particles. We use it here to access the PDG IDs of various $B$-hadrons, and check which particles in our sample have that PDG ID. 

We access the values through the `literals` module of `particle`, which provides shortcuts to lots of common particles. So doing `lp.B_0.pdgid` gives us the PDG ID of the $B^0$ meson. A full list of these literals can be found [here](https://github.com/scikit-hep/particle/blob/master/src/particle/shared_literals.py).

Here we loop over some different $B$-hadrons, defined in the `b_types` dictionary. This dictionary contains the literal for a given $B$-hadron from the `particle` package, and also a LaTeX name we'll use later.

In [4]:
b_types = {"Bd": [lp.B_0,"$B^0$"], 
           "Bu": [lp.B_plus,"$B^\pm$"], 
           "Bs": [lp.B_s_0,"$B_s^0$"], 
           "Bc": [lp.B_c_plus,"$B_c^\pm$"],
           "Lb": [lp.Lambda_b_0,"$\\Lambda_b^0$"],
          }
n = {}
for b in b_types:
    b_cut = abs(gen[gen_container,"pdgId"]) == abs(b_types[b][0].pdgid)
    n[b] = ak.sum(ak.flatten(b_cut))
print(n)

{'Bd': 1058, 'Bu': 859, 'Bs': 349, 'Bc': 3, 'Lb': 70}


Here we have used `ak.flatten()` to turn the jagged array (more than one particle per event) into a flat array. We then use the `ak.sum()` function to count the total number of each $B$-hadron species across all events.

We want to know what fraction of the $b$-quarks hadronise to each of the $B$-hadrons. To calculate this, we need the total number of $b$-quarks produced. This one is easy: since we are dealing with $Z^0 \to b\bar{b}$, every event contains 2 $b$-quarks. So the total number of $b$-quarks is just twice the total number of events in our `ROOT` file. 

In [5]:
n_bbbar = 2*tree.num_entries

Let's calculate the production fraction for each $B$-hadron:

In [6]:
prod_frac = {}
for b in b_types:
    prod_frac[b] = 100*(n[b] / n_bbbar)
print(prod_frac)

{'Bd': 52.900000000000006, 'Bu': 42.95, 'Bs': 17.45, 'Bc': 0.15, 'Lb': 3.5000000000000004}


## Persisting the results

We can store these values in a dictionary and write it to a `.json` file. `JSON` format is great whenever you need to re-load some analysis numbers you have calculated within another script - it is an important form of analysis persistence.

In [7]:
with open(f'{loc.JSON}/b_hadron_prod_fracs.json', 'w') as f:
    json.dump(prod_frac, f, sort_keys=True, indent=4)

Let's finish off by making a LaTeX summary table of our production fractions. We can write this to a `.tex` file which we can then put into some documentation or Beamer slides.

In [8]:
f_tex = open(f'{loc.TABLES}/b_hadron_prod_fracs.tex','w')
f_tex.write('\\begin{table}[!h] \n')
f_tex.write('\\centering \n')
f_tex.write('\\begin{tabular}{l c} \n')
f_tex.write('$B$-hadron & Production fraction (\%) \\\\ \\hline \n')
for b in b_types:
    f_tex.write(f'{b_types[b][1]} & {prod_frac[b]:.2f} \\\\ \n')
f_tex.write('\\end{tabular} \n')
f_tex.write('\\caption{$B$-hadron production fractions in $Z^0 \\to b\\bar{b}$ simulation. \\label{tab:b_hadron_prod_fracs}} \n')
f_tex.write('\\end{table}')
f_tex.close()

Note the use of double backslash (`\\`) here, which is required in order for the `write()` command to interpret the backslash properly and print it. The use of `\n` is different, as this is the command to print on a new line next time `write()` is called. 